In [15]:
# Verification cell - run this first
import sys
import os

print(f"Python executable: {sys.executable}")
print(f"Python version: {sys.version}")

# Check if crewai is available
try:
    from crewai.tools import BaseTool
    print("✅ crewai.tools imported successfully!")
except ImportError as e:
    print(f"❌ Import error: {e}")

# Check environment path
venv_path = os.environ.get('VIRTUAL_ENV', 'Not set')
print(f"Virtual environment: {venv_path}")

Python executable: /Users/cameronbell/Projects/agents/.venv/bin/python
Python version: 3.12.9 (main, Feb  4 2025, 14:38:38) [Clang 16.0.0 (clang-1600.0.26.6)]
✅ crewai.tools imported successfully!
Virtual environment: /Users/cameronbell/Projects/agents/.venv


In [16]:
import os
import requests
import datetime as dt
from typing import Optional
from pydantic import BaseModel, Field
from crewai.tools import BaseTool

### from financial_data_tool_v2.py

In [17]:
# --- Pydantic Output Model for the Tool ---
class MarketSnapshotV2(BaseModel):
    """Real-time data for a single asset (the A-Model)."""
    symbol: str = Field(description="The ticker symbol of the asset.")
    timestamp_utc: dt.datetime = Field(description="The timestamp of the snapshot, standardized to UTC.")
    last_price: float = Field(description="The last traded price of the asset.")
    prev_close: Optional[float] = Field(description="The closing price from the previous trading day.")
    volume: int = Field(description="Current session volume (intraday).")

In [37]:
# --- The CrewAI Tool Class ---
class FinancialDataToolV2(BaseTool):
    name: str = "Financial Data Tool V2"
    description: str = (
        "Fetches the current real-time market snapshot for a given stock ticker, "
        "including last price and current volume from the Marketstack Real-Time endpoint. "
        "Goal: pricing, volume + volatility data."
    )

    def _run(self, ticker: str) -> MarketSnapshotV2:
        """
        Fetches the current market snapshot data from the Marketstack API.

        Args:
            ticker (str): The stock ticker symbol (e.g., 'AAPL').

        Returns:
            MarketSnapshotV2 | str: The structured real-time market data or an error string.
        """
        ACCESS_KEY = os.environ.get("MARKETSTACK_API_KEY")
        # Using the End of Day (EOD) endpoint - returns historical data sorted by date DESC
        BASE_URL = "https://api.marketstack.com/v2/eod" 
        
        params = {
            'access_key': ACCESS_KEY,
            'symbols': ticker,
        }
        
        try:
            response = requests.get(BASE_URL, params=params)
            response.raise_for_status()
            data = response.json().get('data', [])
            
            if not data:
                return f"Error: No data returned for ticker {ticker}. Check symbol validity."

            # Data is already sorted DESC by date (newest first)
            # Get the most recent day's data
            latest_snapshot = data[0]
            
            # Get previous day's close if available (data[1])
            prev_close_value = None
            if len(data) > 1:
                prev_close_value = data[1].get('close')

            # Helper to safely cast
            def safe_float(value):
                return float(value) if value is not None else None
            def safe_int(value):
                return int(value) if value is not None else 0

            return MarketSnapshotV2(
                symbol=latest_snapshot.get('symbol'),
                timestamp_utc=dt.datetime.fromisoformat(latest_snapshot.get('date').replace('Z', '+00:00')),
                last_price=safe_float(latest_snapshot.get('close')),  # Use 'close' instead of 'last'
                prev_close=safe_float(prev_close_value),  # Get from data[1] if available
                volume=safe_int(latest_snapshot.get('volume')),
            )

        except requests.exceptions.RequestException as e:
            return f"API Request Error for {ticker}: {e}"
        except Exception as e:
            return f"An unexpected error occurred while processing real-time data for {ticker}: {e}"

### from financial_data_tool.py

In [47]:
# --- Pydantic Output Model for the Tool ---
class MarketSnapshot(BaseModel):
    """Real-time data for a single asset (the A-Model)."""
    symbol: str = Field(description="The ticker symbol of the asset.")
    exchange: str = Field(description="The exchange the asset trades on (e.g., XNAS).")
    timestamp_utc: dt.datetime = Field(description="The timestamp of the snapshot, standardized to UTC.")
    last_price: float = Field(description="The last traded price of the asset.")
    pct_change_1d: float = Field(description="Percentage change (e.g., 0.015 for +1.5%) vs previous close.")
    volume: int = Field(description="Current session volume (intraday).")
    prev_close: Optional[float] = Field(description="The closing price from the previous trading day.")
    day_high: Optional[float] = Field(description="Highest price reached during the current trading day.")
    day_low: Optional[float] = Field(description="Lowest price reached during the current trading day.")


In [49]:
# --- The CrewAI Tool Class ---
class FinancialDataTool(BaseTool):
    name: str = "Financial Data Tool"
    description: str = (
        "Fetches the current real-time market snapshot for a given stock ticker, "
        "including last price, volume, and daily high/lows. Required for immediate risk assessment."
    )

    def _run(self, ticker: str) -> MarketSnapshot:
        """
        Fetches the current market snapshot data from the Marketstack API.

        Args:
            ticker (str): The stock ticker symbol (e.g., 'AAPL').

        Returns:
            MarketSnapshot: The structured real-time market data.
        """
        ACCESS_KEY = os.environ.get("MARKETSTACK_API_KEY")
        BASE_URL = "https://api.marketstack.com/v2/eod"
        
        params = {
            'access_key': ACCESS_KEY,
            'symbols': ticker,
        }
        
        try:
            response = requests.get(BASE_URL, params=params)
            response.raise_for_status()
            data = response.json().get('data', [])
            
            if not data:
                return f"Error: No data returned for ticker {ticker}. Check symbol validity."

            snapshot = data[0]

        
            # Helper to safely convert to float/int, defaulting to None/0 if missing
            def safe_cast(value, default_type):
                if value is None:
                    return None if default_type == float else 0
                try:
                    return default_type(value)
                except (ValueError, TypeError):
                    return None if default_type == float else 0

            # Calculate pct_change_1d
            prev_close = safe_cast(data[1].get('close'), float)
            last_price = safe_cast(snapshot.get('close'), float)

            pct_change_1d = 0.0
            if prev_close and last_price and prev_close != 0:
                pct_change_1d = (last_price - prev_close) / prev_close
            
            # NOTE: Marketstack uses 'last' for last price, 'volume' for current session volume,
            # 'high'/'low' for day high/low.
            
            return MarketSnapshot(
                symbol=snapshot.get('symbol'),
                exchange=snapshot.get('exchange'),
                timestamp_utc=dt.datetime.fromisoformat(snapshot.get('date').replace('Z', '+00:00')),
                last_price=last_price,
                pct_change_1d=pct_change_1d,
                volume=safe_cast(snapshot.get('volume'), int),
                prev_close=prev_close,
                day_high=safe_cast(snapshot.get('high'), float),
                day_low=safe_cast(snapshot.get('low'), float),
            )

        except requests.exceptions.RequestException as e:
            return f"API Request Error for {ticker}: {e}"
        except Exception as e:
            return f"An unexpected error occurred while processing data for {ticker}: {e}"

### suggested edits to class in financial_data_tool.py

In [51]:
# --- The CrewAI Tool Class ---
class FinancialDataTool(BaseTool):
    name: str = "Financial Data Tool"
    description: str = (
        "Fetches the current real-time market snapshot for a given stock ticker, "
        "including last price, volume, and daily high/lows. Required for immediate risk assessment."
    )

    def _run(self, ticker: str) -> MarketSnapshot:
        """
        Fetches the current market snapshot data from the Marketstack API.

        Args:
            ticker (str): The stock ticker symbol (e.g., 'AAPL').

        Returns:
            MarketSnapshot: The structured real-time market data.
        """
        ACCESS_KEY = os.environ.get("MARKETSTACK_API_KEY")
        BASE_URL = "https://api.marketstack.com/v2/eod"
        
        params = {
            'access_key': ACCESS_KEY,
            'symbols': ticker,
            'limit': 2  # Get 2 days: [0] = latest, [1] = previous
        }
        
        try:
            response = requests.get(BASE_URL, params=params)
            response.raise_for_status()
            data = response.json().get('data', [])
            
            if not data:
                return f"Error: No data returned for ticker {ticker}. Check symbol validity."

            # Get latest snapshot
            latest_snapshot = data[0]
            
            # Get previous day's close if available
            prev_close_value = None
            if len(data) > 1:
                prev_close_value = data[1].get('close')
            
            # Helper to safely convert to float/int, defaulting to None/0 if missing
            def safe_cast(value, default_type):
                if value is None:
                    return None if default_type == float else 0
                try:
                    return default_type(value)
                except (ValueError, TypeError):
                    return None if default_type == float else 0

            # Calculate pct_change_1d
            last_price = safe_cast(latest_snapshot.get('close'), float)
            prev_close = safe_cast(prev_close_value, float)
            pct_change_1d = 0.0
            if prev_close and last_price and prev_close != 0:
                pct_change_1d = (last_price - prev_close) / prev_close
            
            return MarketSnapshot(
                symbol=latest_snapshot.get('symbol'),
                exchange=latest_snapshot.get('exchange'),
                timestamp_utc=dt.datetime.fromisoformat(latest_snapshot.get('date').replace('Z', '+00:00')),
                last_price=last_price,
                pct_change_1d=pct_change_1d,
                volume=safe_cast(latest_snapshot.get('volume'), int),
                prev_close=prev_close,
                day_high=safe_cast(latest_snapshot.get('high'), float),
                day_low=safe_cast(latest_snapshot.get('low'), float),
            )

        except requests.exceptions.RequestException as e:
            return f"API Request Error for {ticker}: {e}"
        except Exception as e:
            return f"An unexpected error occurred while processing data for {ticker}: {e}"

Changes needed in FinancialDataTool: <br>
Line 25: Add 'limit': 2 to params <br>
Line 35: Change snapshot = data[0] to latest_snapshot = data[0]<br>
Line 47: Change snapshot.get('close_yesterday') to get from data[1] if available<br>
Line 48: Change snapshot.get('last') to latest_snapshot.get('close')<br>
Line 56: Update all snapshot.get() calls to latest_snapshot.get()<br>
After fixing the class, run the test script to see formatted results for all fields.<br>

In [38]:
# Cell 4: Test the tool
# Create an instance and test it
tool = FinancialDataToolV2()

# Test with AAPL
result = tool._run('AAPL')
print(result)
print(f"\nType: {type(result)}")
if isinstance(result, MarketSnapshotV2):
    print(f"Symbol: {result.symbol}")
    print(f"Last Price: ${result.last_price}")
    print(f"Volume: {result.volume:,}")
    print(f"Previous Close: ${result.prev_close}")

symbol='AAPL' timestamp_utc=datetime.datetime(2025, 11, 14, 0, 0, tzinfo=datetime.timezone.utc) last_price=272.41 prev_close=272.95 volume=47399300

Type: <class '__main__.MarketSnapshotV2'>
Symbol: AAPL
Last Price: $272.41
Volume: 47,399,300
Previous Close: $272.95


In [50]:
# Test with AAPL
tool1 = FinancialDataTool()
result = tool1._run('AAPL')
print(result)
print(f"\nType: {type(result)}")
if isinstance(result, MarketSnapshot):
    print(f"Symbol: {result.symbol}")
    print(f"Last Price: ${result.last_price}")
    print(f"Volume: {result.volume:,}")
    print(f"Previous Close: ${result.prev_close}")
    print(f"Day High: ${result.day_high}")
    print(f"Day Low: ${result.day_low}")
    print(f"Pct Change 1d: {result.pct_change_1d}")
    print(f"Exchange: {result.exchange}")
    print(f"Timestamp: {result.timestamp_utc}")


symbol='AAPL' exchange='XNAS' timestamp_utc=datetime.datetime(2025, 11, 14, 0, 0, tzinfo=datetime.timezone.utc) last_price=272.41 pct_change_1d=-0.0019783843194722977 volume=47399300 prev_close=272.95 day_high=275.96 day_low=269.6

Type: <class '__main__.MarketSnapshot'>
Symbol: AAPL
Last Price: $272.41
Volume: 47,399,300
Previous Close: $272.95
Day High: $275.96
Day Low: $269.6
Pct Change 1d: -0.0019783843194722977
Exchange: XNAS
Timestamp: 2025-11-14 00:00:00+00:00


In [52]:
# ============================================
# Comprehensive Test Script for FinancialDataTool
# ============================================

def test_financial_data_tool(ticker='AAPL', verbose=True):
    """
    Test the FinancialDataTool with a given ticker.
    
    Args:
        ticker: Stock ticker symbol to test
        verbose: If True, print detailed output
    """
    print("=" * 60)
    print(f"Testing FinancialDataTool with ticker: {ticker}")
    print("=" * 60)
    
    # Check API key
    api_key = os.environ.get("MARKETSTACK_API_KEY")
    if not api_key:
        print("⚠️  WARNING: MARKETSTACK_API_KEY not set in environment")
    else:
        print(f"✅ API Key found: {api_key[:10]}...")
    
    print("\n1. Creating tool instance...")
    tool = FinancialDataTool()
    print(f"   Tool Name: {tool.name}")
    print(f"   Tool Description: {tool.description}")
    
    print(f"\n2. Fetching data for {ticker}...")
    result = tool._run(ticker)
    
    print(f"\n3. Result Analysis:")
    print("-" * 60)
    
    # Check if result is an error string or MarketSnapshot object
    if isinstance(result, str):
        print(f"❌ ERROR: {result}")
        return None
    elif isinstance(result, MarketSnapshot):
        print("✅ Success! Received MarketSnapshot object")
        print("\n4. Market Data:")
        print("-" * 60)
        print(f"   Symbol:           {result.symbol}")
        print(f"   Exchange:         {result.exchange}")
        print(f"   Timestamp (UTC):  {result.timestamp_utc}")
        print(f"   Last Price:       ${result.last_price:,.2f}")
        print(f"   Previous Close:   ${result.prev_close:,.2f if result.prev_close else 'N/A'}")
        print(f"   Day High:         ${result.day_high:,.2f if result.day_high else 'N/A'}")
        print(f"   Day Low:          ${result.day_low:,.2f if result.day_low else 'N/A'}")
        print(f"   Volume:           {result.volume:,}")
        print(f"   % Change (1d):   {result.pct_change_1d*100:+.2f}%")
        
        # Calculate additional metrics
        if result.prev_close:
            price_change = result.last_price - result.prev_close
            print(f"\n5. Calculated Metrics:")
            print("-" * 60)
            print(f"   Price Change:     ${price_change:,.2f}")
            print(f"   % Change:         {(price_change/result.prev_close)*100:+.2f}%")
        
        if result.day_high and result.day_low:
            day_range = result.day_high - result.day_low
            range_pct = (day_range / result.last_price) * 100
            print(f"   Day Range:        ${day_range:,.2f} ({range_pct:.2f}%)")
        
        return result
    else:
        print(f"⚠️  Unexpected result type: {type(result)}")
        print(f"   Result: {result}")
        return None

# Run the test
print("\n" + "=" * 60)
print("FINANCIAL DATA TOOL TEST")
print("=" * 60 + "\n")

# Test with AAPL
result_aapl = test_financial_data_tool('AAPL')

# Test with multiple tickers (optional)
print("\n\n" + "=" * 60)
print("MULTI-TICKER TEST")
print("=" * 60 + "\n")

test_tickers = ['AAPL', 'MSFT', 'GOOGL']
results = {}

for ticker in test_tickers:
    print(f"\n{'='*60}")
    result = test_financial_data_tool(ticker, verbose=False)
    results[ticker] = result
    print()

# Summary
print("\n" + "=" * 60)
print("TEST SUMMARY")
print("=" * 60)
for ticker, result in results.items():
    if isinstance(result, MarketSnapshot):
        status = "✅"
        price = f"${result.last_price:,.2f}"
    else:
        status = "❌"
        price = "N/A"
    print(f"{status} {ticker:6s}: {price}")


FINANCIAL DATA TOOL TEST

Testing FinancialDataTool with ticker: AAPL
✅ API Key found: b825c135fa...

1. Creating tool instance...
   Tool Name: Financial Data Tool
   Tool Description: Tool Name: Financial Data Tool
Tool Arguments: {'ticker': {'description': None, 'type': 'str'}}
Tool Description: Fetches the current real-time market snapshot for a given stock ticker, including last price, volume, and daily high/lows. Required for immediate risk assessment.

2. Fetching data for AAPL...

3. Result Analysis:
------------------------------------------------------------
✅ Success! Received MarketSnapshot object

4. Market Data:
------------------------------------------------------------
   Symbol:           AAPL
   Exchange:         XNAS
   Timestamp (UTC):  2025-11-14 00:00:00+00:00
   Last Price:       $272.41


ValueError: Invalid format specifier ',.2f if result.prev_close else 'N/A'' for object of type 'float'